<center><p float="center">
  <img src="https://mma.prnewswire.com/media/1458111/Great_Learning_Logo.jpg?p=facebook" width="200" height="100"/>
</p></center>                                                                         <center><h1></h1></center>
<h1><left> <font size=5>Project</font></left></h1>

# Business Context

## Implementing a RAG System for Power BI Usage

**Problem Scenario:**

In the current data-driven landscape, organizations increasingly rely on powerful analytics tools like Power BI to derive insights and make informed decisions. However, many analysts struggle with the complexity and breadth of Power BI’s official documentation. The extensive resources often lead to confusion, causing users to misinterpret features or overlook essential functionalities. This challenge can result in inefficient data analysis, wasted time, and missed opportunities for actionable insights. Consequently, analysts may not fully leverage the capabilities of Power BI, stifling potential business growth and impact.

# Objective

To address these challenges, we propose implementing a **Retrieval-Augmented Generation (RAG) system** specifically designed for Power BI. This system will enable analysts to formulate questions using natural language and retrieve concise, relevant answers directly from the official documentation. By facilitating better access to critical information, we aim to enhance the operational efficiency of analysts and empower them to utilize Power BI to its fullest potential.

The RAG application will simplify interactions with Power BI documentation, allowing users to inquire about specific features, functions, or best practices and receive clear explanations in real-time. By improving understanding and accessibility to the tool, analysts will be able to make quicker, data-driven decisions that lead to a significant business impact.

# **<font color=blue> Setting up OpenAI API Key**

In [1]:
import os
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Installing and Importing the Necessary Libraries

In this section, we need to install and import libraries required to run the notebook:

- The `openai` package provides the official OpenAI API client for accessing models like GPT-4, Whisper, DALL·E, including its embedding models

- The `tiktoken`	library provides access to OpenAI's tokenizer models, crucial for chunking and token counting

- The `pypdf` library parses and extracts text from PDF files — useful for document ingestion

- LangChain is a GenAI framework to build applications with LLMs using chains and agents.
  - `langchain` is the core library that provides access to various LangChain abstractions
  - `langchain-community` provides access to 3rd-party integrations (e.g., different vector stores, tools)
  - `langchain-chroma` provides specific integration to use ChromaDB as the vector store backend in LangChain
  - `langchain-openai` module provides a plug-in interface for LangChain to call OpenAI's LLMs using standardized interface

- `chromadb` library provides access to ChromaDB vector database, which is a fast, vector database optimized for retrieval in RAG systems

In [3]:
# Installing the required libraries
# Installing the required libraries
!pip install -q openai==1.66.3 \
                tiktoken==0.9.0 \
                pypdf==5.4.0 \
                langchain==0.3.20 \
                langchain-community==0.3.19 \
                langchain-chroma==0.2.2 \
                langchain-openai==0.3.9 \
                chromadb==0.6.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 19.1 MB/s eta 0:0

**Importing the Libraries**


In [4]:
# Importing the standard Libraries
import time                           # For measuring execution time or adding delays
from datetime import datetime         # For handling timestamps and datetime operations

# ChromaDB Vector Database
import chromadb  # Chroma: a local-first vector database for storing and querying document embeddings

# OpenAI SDK
from openai import OpenAI             # Official OpenAI Python SDK (v1.x) for interacting with models like GPT-4

# LangChain Utilities
# RecursiveCharacterTextSplitter intelligently breaks long text into smaller chunks with some overlap, preserving context.
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Loads all PDF files from a directory and extracts text from each.
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.document_loaders import PyPDFLoader

# Base class representing a document in LangChain; useful for downstream chaining and processing.
from langchain_core.documents import Document

# Embeddings and Vector Store
# Generates vector embeddings using OpenAI’s embedding models (e.g., `text-embedding-3-small`)
from langchain_openai import OpenAIEmbeddings

# Integration for using Chroma as the vector store within LangChain’s ecosystem
from langchain_chroma import Chroma

## Setup the API Key
#### Setup the OpenAI API key and initialize the client with the required model.

In [5]:
# Accessing environment variables via Colab Secrets
from google.colab import userdata

# Set the OpenAI key in Colab Secrets
openai_api_key = userdata.get('OPENAI_API_KEY') # Replace with the appropriate secret key name

# Refer to the content page present at the top of module for accessing the Open AI API_KEY from Olympus platform
# The following code sets up the OpenAI client instance with the API Key and the API Endpoint
client = OpenAI(
    api_key = openai_api_key,
    base_url = "openai_base_url" # OpenAI Endpoint Base URL
)

# GPT-4o-mini will be the primary model used in this notebook
model_name = 'gpt-4o-mini'

## Creating Vector Database

In [10]:
# Unzip the dataset containing the policy document
!unzip /PowerBI.zip

Archive:  /PowerBI.zip
  inflating: Introducing_Power_BI.pdf  


## Load PDF Documents and perform chunking


In [11]:
# Set the directory where PDF files to be stored

pdf_folder_location = "/content/Introducing_Power_BI.pdf"
pdf_loader = PyPDFLoader(pdf_folder_location)


In [12]:
docs = pdf_loader.load()
first_page_text = docs[1].page_content
print(first_page_text)

PUBLISHED BY 
Microsoft Press 
A division of Microsoft Corporation 
One Microsoft Way 
Redmond, Washington 98052-6399 
Copyright © 2016 by Microsoft Corporation 
All rights reserved. No part of the contents of 
this book may be reproduced or transmitted in 
any form or by any means without the written 
permission of the publisher. 
ISBN: 978-1-5093-0228-4 
Microsoft Press books are available through 
booksellers and distributors worldwide. If you 
need support related to this book, email 
Microsoft Press Support at 
mspinput@microsoft.com. Please tell us what 
you think of this book at http://aka.ms/tellpress. 
This book is provided “as-is” and expresses the 
author’s views and opinions. The views, opinions 
and information expressed in this book, 
including URL and other Internet website 
references, may change without notice. 
Some examples depicted herein are provided for 
illustration only and are fictitious. No real


In [13]:
# Chunks are stored within LangChain's Document class
#enc = tiktoken.encoding_for_model(model_name)
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name= "cl100k_base",
    chunk_size=512,
    chunk_overlap=16
)

powerbi_chunks = pdf_loader.load_and_split(text_splitter)

In [ ]:
#enc

<Encoding 'o200k_base'>

In [14]:
powerbi_chunks[0].page_content

'Introducing\nMicrosoft \nPower BI\nAlberto Ferrari and Marco Russo'

In [15]:
len(powerbi_chunks)

407

In [16]:
# define the ChromaDB collection name to store the chunks
collection_name = "powerbi_docs"

### Initialize the OpenAI embedding model with the API key, endpoint, and embedding model name.


In [17]:
embeddings = OpenAIEmbeddings(
    api_key = openai_api_key,
    base_url = "openai_base_url",
    model="text-embedding-3-small"
)

In [18]:
# Initialize a persistent Chroma client
chromadb_client = chromadb.PersistentClient(
    path="./powerbi_docs")


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


In [19]:
chromadb_client.heartbeat()

1761170367830208873

In [20]:
chromadb_client.count_collections()

0

In [21]:
# Instantiate a Chroma vector store to store and retrieve document embeddings
vectorstore = Chroma(
    client=chromadb_client,
    collection_name=collection_name,
    collection_metadata = {"hnsw:space": "cosine"},
    embedding_function = embeddings,
    persist_directory = "./powerbi_docs"
)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [22]:
# confirm collection creation
chromadb_client.list_collections()

['powerbi_docs']

In [23]:
# Confirm database has been populated with the collection
chromadb_client.count_collections()

1

In [24]:
# Batch 500 chunks to send to the API at a time, pausing execution for 30 seconds afterward
batch_size = 500
for i in range(0, len(powerbi_chunks), batch_size):
    batch = powerbi_chunks[i:i+batch_size]
    batch_ids = [f"text_{idx}" for idx in range(i, i + len(batch))]
    vectorstore.add_documents(batch, ids = batch_ids)
    print(f"Processed batch {i//batch_size + 1}, pausing for 30 seconds...")
    time.sleep(30)

Processed batch 1, pausing for 30 seconds...


# CRUD Operations in ChromaDB




## **READ**

Once the database is created, the stored entries can be retrieved by initializing a new Chroma instance (denoted as **vectorstore_persisted** to distinguish between creation and read operations) and directing it to the persistent storage directory containing the document embeddings.

In this step, you need to:

Initialize a new Chroma instance (e.g., vectorstore_persisted) and point it to the persistent storage directory where embeddings are stored.

In [25]:
vectorstore_persisted = Chroma(
    collection_name = collection_name,
    collection_metadata = {"hnsw:space": "cosine"},
    embedding_function = embeddings,
    client = chromadb_client,
    persist_directory = "./powerbi_docs"
)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


There are two valuable types of READ operations in vector databases:

1. **Inspecting individual records**
2. **Retrieving relevant records based on a user query**


**Inspecting individual records**

In [26]:
# Define the chroma collection
individual_records = chromadb_client.get_collection(collection_name)

In [27]:
# Count the number of records in the collection
individual_records.count()

407

In [28]:
# Inspect the first 3 records using the .peek() method
individual_records.peek(3)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionGetEvent: capture() takes 1 positional argument but 3 were given


{'ids': ['text_0', 'text_1', 'text_2'],
 'embeddings': array([[-0.0141233 , -0.03104495,  0.0387464 , ..., -0.00077508,
          0.00032513,  0.01716083],
        [ 0.0097238 ,  0.00484162,  0.02150327, ..., -0.019948  ,
         -0.01871731,  0.01614773],
        [-0.01451573, -0.00566878,  0.01017629, ..., -0.00789198,
         -0.00434708,  0.00459538]]),
 'documents': ['Introducing\nMicrosoft \nPower BI\nAlberto Ferrari and Marco Russo',
  'PUBLISHED BY \nMicrosoft Press \nA division of Microsoft Corporation \nOne Microsoft Way \nRedmond, Washington 98052-6399 \nCopyright © 2016 by Microsoft Corporation \nAll rights reserved. No part of the contents of \nthis book may be reproduced or transmitted in \nany form or by any means without the written \npermission of the publisher. \nISBN: 978-1-5093-0228-4 \nMicrosoft Press books are available through \nbooksellers and distributors worldwide. If you \nneed support related to this book, email \nMicrosoft Press Support at \nmspinput@micr

In [29]:
individual_records.get(
    ids = ['text_50']
)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionGetEvent: capture() takes 1 positional argument but 3 were given


{'ids': ['text_50'],
 'embeddings': None,
 'documents': ['33 C H A P T E R  1  |  Introducing Power BI \n \nfilter is activated by simply clicking the chart. So \nfar, you have seen that a filter—when applied to \nother charts—highlights the relative contribution \nof the filtered item against the grand total by \nusing two colors. This behavior is known as \nvisual interaction, and it is extremely interesting. \nYet, there are scenarios, like the one David is \nexperimenting with, for which it would be better \nto compare the differences between \ncountries/regions more than the overall \ncontribution of a brand against the other brands. \nYou can configure visual interactions in a highly \nprecise way. Namely, you can configure how the \nfiltering on a chart behaves with respect to all of \nthe other ones. The scenario we are looking at—\nwith only two charts—\nis perfect for \nexperimenting because it is very simple. To \nconfigure visual interactions, on the top menu \nbar of the r

**Retrieving relevant records based on a user query**

The primary function of the vector database is to retrieve relevant records based on user queries and to facilitate this process, we implement a retriever that utilizes the query embeddings to query the database.

Write code that uses HNSW algorithm to calculate the nearest neighbors for the user query and returns the corresponding documents from the database.

In [30]:
# Create a retriever interface from the vector store
retriever = vectorstore_persisted.as_retriever(
    search_type = 'similarity',             # Use the default method based on semantic similarity
    search_kwargs = {'k': 5}                # Retrieve top 5 most similar chunks
)

# Define a sample user query
user_query = "Give introduction to PowerBi"

Write code to  performs the similarity search based on the user query by using the `.invoke()` method.

In [31]:
retriever.invoke(user_query)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


[Document(id='text_29', metadata={'author': 'Joan', 'creationdate': '2016-06-13T10:18:21-04:00', 'creator': 'Adobe Acrobat Pro 10.1.16', 'moddate': '2016-06-13T21:13:38-04:00', 'page': 29, 'page_label': '30', 'producer': 'Adobe Acrobat Pro 10.1.16', 'source': '/content/Introducing_Power_BI.pdf', 'title': '', 'total_pages': 407}, page_content='12 C H A P T E R  1  |  Introducing Power BI \n \nBefore going any further, we want to take a few \nmoments to explain how the Power BI portal is \norganized. On the left side of the screen, in the \npane labeled My Workspace, there are several \nitems. Let’s take a look at them: \n\uf0b7 Dashboards This lists all of the dashboards \nyou have created. After loading a single \nworkbook, Power BI creates a dashboard for \nyou, using the same name as that of the \noriginal workbook. \n\uf0b7 Reports Here, you will see the reports \nbased on your data. In Figure 1-7, there is no \ndefault report, but we’ll follow along as \nDavid creates one very soon

## Observations / Learnings
* Uniziped filed using PyPDFLoader and not PyPDFDirectoryLoader since we had only one file in the zipped folder.
* Verfied the chunks using .page_content
* Tried using enc = tiktoken.encoding_for_model(model_name) in text_splitter, but it is choosing o200k_base encoding. So hardcoded it to standard cl100k_base encoding.
* Split text with chunk size = 512 and overlap = 16. It basically took one page for each chunk.I don't see any overlap between the chunks because overlap does not work for different pages. If we have 2 chunks from the same page, we would be able to see the overlap.
* We need to upload the zip file everytime when the notebook disconnects to the terminal.
* Learned to provide stable IDs (text_{i}) to avoid duplicates. It was difficult to do .peek() without stable IDs because it generated random IDs and to verify with the documnet was difficult.
* Since we are using k = 5 in retriever, top 5 relevant chunks are returned.

# RAG Q&A System for PowerBI Documentation

A typical RAG implementation consists of the following stages:
* Indexing Stage
* Retrieval Stage
* Generation Stage

| Stage          | Key Activities                                        | Role in RAG                              |
| -------------- | ----------------------------------------------------- | ---------------------------------------- |
| **Indexing**   | Chunking · Embedding · Storing                        | Prepares data for efficient retrieval    |
| **Retrieval**  | Query embedding · Similarity search   | Consolidates relevant context            |
| **Generation** | Prompt construction · LLM generation | Produces final response grounded in data |


Let's now put together the RAG pipeline using these stages.



## Retrieval Stage

**Retrieving Relevant Documents**

Write code that performs the Retrieval stage in the RAG pipeline.

 define a sample user query to test the RAG pipeline

In [32]:
sample_user_query = "How to connect PowerBi to a database?"

Retrieve the relevant chunks from the documents based on the `user_query`.


In [33]:
relevant_document_chunks = retriever.invoke(sample_user_query)

In [34]:
len(relevant_document_chunks)

5

In [35]:
# Inspecting the first document
for document in relevant_document_chunks:
    print(document.page_content.replace("\t", " "))
    break

137 C H A P T E R  4  |  Using Power BI Desktop 
 
Figure 4-3: Before importing data from an SQL 
database, you need to choose the loading method. 
Let’s take a moment to learn about this 
connection option because it is an important 
one and will help shed more light on how Power 
BI connections work. 
When you choose Import, Power BI Desktop 
connects to the database, loads the information, 
and stores it within its internal data model. You 
can then work on your data in Power BI Desktop 
without being connected to the database. You 
will only need a connection when you want to 
refresh the data. 
With DirectQuery, Power BI Desktop does not 
load the data into its internal database. Instead, 
it runs a query to the original database every 
time it needs to draw a chart or, in general, run a 
query. Thus, the connection between Power BI 
Desktop and the database will be permanent. 
The contrast in the query timings reflects a key 
difference: when you use Import, you are 
working with

## Generation Stage

This section will perform the **Generation** stage of the RAG pipeline.

We will pass the relevant context chunks to the LLM, along with the system message and user message via a prompt template.

These are then passed to the LLM to compose an appropriate response to the user's query.


### Prompt Template

Define the system message for the RAG chatbot with the appopriate role, context and the relevant instructions.

In [36]:
qna_system_message = """
You are an assistant to analytics team who answers user queries on PowerBi.
User input will have the context required by you to answer user queries.
This context will be delimited by:
<Context> and </Context>.
The context contains references to specific portions of a document relevant to the user query.
User queries will be delimited by:
<Question> and </Question>.
Please answer user queries only using the context provided in the input.
Do not mention anything about the context in your final answer. Your response should only contain the answer to the question.
If the answer is not found in the context, respond "I don't know".
"""

Write the user message prompt template that provides the relevant chunks and the user query within the `context` and `question` placeholders respectively.

In [37]:
qna_user_message_template = """
<Context>
Here are some documents that are relevant to the question mentioned below.
{context}
</Context>

<Question>
{question}
</Question>
"""

### Generating the Response
In this step, you need to:   

Prompt construction, LLM API call with error handling, and response parsing

In [38]:
user_query = "How to configure automatic refresh in PowerBI?"

relevant_document_chunks = retriever.invoke(user_query)
context_list = [d.page_content for d in relevant_document_chunks]
context_for_query = "\n---\n".join(context_list)

prompt = [
    {'role': 'system', 'content': qna_system_message},
    {'role': 'user', 'content': qna_user_message_template.format(
         context=context_for_query,
         question=user_query
        )
    }
]

try:
    response = client.chat.completions.create(
        model=model_name,
        messages=prompt,
        temperature=0
    )

    prediction = response.choices[0].message.content.strip()
except Exception as e:
    prediction = f'Sorry, I encountered the following error: \n {e}'

print(prediction)

To configure automatic refresh in Power BI, you need to expand the Schedule Refresh section after setting the credentials for your data source. This allows you to define when Power BI attempts to refresh the dataset. You can schedule the refresh to run daily or weekly at different times. Note that a Power BI Pro license is required to schedule more than one refresh per day; with a free license, you can only schedule one daily refresh.


## Observation

*  If we give strict prompt with context and questions, it only answers from the context. And if the answer is not found in the context, it responds "I don't know" as given.
*   It helps in reducing hallucination risk.
*   Gives facts based answers only. (basically only from the uploaded document)

# Putting it all together - PowerBI RAG Q&A Chatbot

We'll now put together the relevant codes for the RAG pipeline into a file named `rag-chat.py` to create a basic command-line chat interface which can run via  the terminal.

This naive RAG implementation illustrates how document Q&A could be automated for any domain.

Write code that use the `%%writefile` magic command specific to Google Colab, which allows the content of a cell to be written directly into a file on the virtual machine's disk.

This allows for the creation of scripts, configuration files, or data files within the Colab environment. These files are available during the Colab runtime and are deleted when the runtime is stopped or deleted.

The `!python` shell command can be used to execute a Python script (.py files) or commands within the Colab environment.

In [39]:
import os
# Set the OpenAI API key as an environment variable.
# This allows libraries (like the OpenAI SDK) to automatically detect the key without hardcoding it in the script.
os.environ['openai_api_key'] = openai_api_key

In [42]:
%%writefile rag-chat.py
import os
import chromadb

from openai import OpenAI

from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
import logging
logging.getLogger("chromadb").setLevel(logging.CRITICAL)

from google.colab import userdata

model_name = 'gpt-4o-mini'
collection_name = 'powerbi_docs'
openai_api_key = os.environ.get('openai_api_key')

client = OpenAI(
    api_key=openai_api_key,
    base_url="openai_base_url"
)

embedding_model = OpenAIEmbeddings(
    api_key=openai_api_key,
    base_url="openai_base_url",
    model='text-embedding-3-small'
)

chromadb_client = chromadb.PersistentClient(
    path="./powerbi_docs"
)

vectorstore_persisted = Chroma(
    collection_name=collection_name,
    collection_metadata={"hnsw:space": "cosine"},
    embedding_function=embedding_model,
    client=chromadb_client,
    persist_directory="./powerbi_docs"
)

retriever = vectorstore_persisted.as_retriever(
    search_type='similarity',
    search_kwargs={'k': 5}
)

qna_system_message = """
You are an assistant to analytics team who answers user queries on PowerBi.
User input will have the context required by you to answer user queries.
This context will be delimited by:
<Context> and </Context>.
The context contains references to specific portions of a document relevant to the user query.
User queries will be delimited by:
<Question> and </Question>.
Please answer user queries only using the context provided in the input.
Do not mention anything about the context in your final answer. Your response should only contain the answer to the question.
If the answer is not found in the context, respond "I don't know".
"""

qna_user_message_template = """
<Context>
Here are some documents that are relevant to the question mentioned below.
{context}
</Context>

<Question>
{question}
</Question>
"""

def respond(user_query):
    relevant_document_chunks = retriever.invoke(user_query)
    context_list = [d.page_content for d in relevant_document_chunks]
    context_for_query = "\n---\n".join(context_list)

    prompt = [
        {'role': 'system', 'content': qna_system_message},
        {
            'role': 'user', 'content': qna_user_message_template.format(
             context=context_for_query,
             question=user_query)
        }
    ]

    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=prompt,
            temperature=0
        )

        answer = response.choices[0].message.content.strip()
    except Exception as e:
        answer = f'Sorry, I encountered the following error: \n {e}'

    return answer

def main():
    """
    Runs the main interactive loop for the Q&A system.

    This function initializes the conversation history, continuously prompts
    the user for queries, processes the queries using the `respond` function,
    and displays the assistant's responses. It also maintains the
    conversation history for context.

    Args:
        None

    Returns:
        None
    """

    # 1. Initialize conversation history.
    # This list stores the conversation between the user and the assistant.
    # It starts with a system message introducing the assistant's role.
    conversation_history = [
        {'role': 'system', 'content': 'You are a assistant who answers user queries on PowerBi'}

        ]

    # 2. Enter the interactive loop.
    # The loop continues until the user enters 'q' to quit.
    while True:
        # 2.1 Get user input.
        # Prompt the user to enter a query and store it in `user_query`.
        user_query = input("User (type q to quit): ")

        # 2.2 Check for quit condition.
        # If the user enters 'q', break out of the loop.
        if user_query == 'q':
            break

        # 2.3 Process the query and get the answer.
        # Call the `respond` function to process the user query and get the answer.
        answer = respond(user_query)

        # 2.4 Update conversation history.
        # Add the user's query and the assistant's answer to the conversation history.
        conversation_history.append({'role': 'user', 'content': user_query})
        conversation_history.append({'role': 'assistant', 'content': answer})

        # 2.5 Display the assistant's answer.
        # Print the assistant's answer to the console.
        print(f"Assistant: {answer}")

if __name__ == "__main__":
    main()

Writing rag-chat.py


Fomulate 5 queries on the PowerBI Documentation that will then be used to validate the the Q&A RAG Chatbot and provide the output responses.

1. How to share PowerBi dashboard to a group?
2. How to use Query Editor?
3. What is DAX?
4. Is Data refresh and live connection mean the same in PowerBi?
5. How to choose visuals in PowerBi?

Run the script using the `!python` shell command.

In [43]:
!python rag-chat.py

User (type q to quit): How to share PowerBi dashboard to a group?
Assistant: To share a Power BI dashboard with a group, you need to create a group of users in Power BI. Once the group is created, you can automatically share all of your dashboards with that group. However, you must have a Power BI Pro license to access the group feature, as it is not available in the free version. After creating the group, you can invite users to join and provide editing rights to certain users within that group.
User (type q to quit): How to use Query Editor?
Assistant: To use Query Editor in Power BI Desktop, you can follow these steps:

1. Open Power BI Desktop and access the Query Editor by clicking on "Edit Queries" in the Home tab of the ribbon.
2. The Query Editor window will open, displaying various options. The top of the window features a ribbon with four tabs: Home, Transform, Add Column, and View.
3. On the left side, you will see the Query pane, which lists all the queries for your model. 

In [50]:
!jupyter nbconvert --to html "/content/drive/My Drive/Colab_Notebooks/Sukanyaa_Sundaram_Project_1_RAG_Gen_AI_for_Practitioners(Full_Code).ipynb"

[NbConvertApp] Converting notebook /content/drive/My Drive/Colab_Notebooks/Sukanyaa_Sundaram_Project_1_RAG_Gen_AI_for_Practitioners(Full_Code).ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 1 image(s).
[NbConvertApp] Writing 393492 bytes to /content/drive/My Drive/Colab_Notebooks/Sukanyaa_Sundaram_Project_1_RAG_Gen_AI_for_Practitioners(Full_Code).html
